In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.express as px
%matplotlib inline

## Rosbank ML Competition

Ссылка на соревнование: https://boosters.pro/champ_15

Росбанк – часть ведущей международной финансовой группы Societe Generale, банк включен ЦБ РФ в число 11 системно значимых кредитных организаций России. Инновации неотъемлемый процесс работы Росбанка, поэтому активно развивается направленный анализа больших данных.

- Данные

Датасет, который содержит историю транзакций клиентов за 3 месяца льготного использования банковского продукта

- Задача

Задача бинарной классификации – прогноз оттока клиентов

Колонка cl_id содержит вутренний id клиента. Для каждого уникальнго cl_id следует предсказать продолжит ли клиент пользоваться продуктом (target_flag). Значение 0 соответствует отказу, а значение 1 соответствует продолжению использования

In [2]:
raw_df = pd.read_csv(r'D:\\DDDrilll\\ds3-spring-2018\\kaggle\\rosssss\\rossbank.csv')
raw_df.head(5)
# target_sum - можно выкинуть, переменная участвует в другой задаче

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   PERIOD        490513 non-null  object 
 1   cl_id         490513 non-null  int64  
 2   MCC           490513 non-null  int64  
 3   channel_type  487603 non-null  object 
 4   currency      490513 non-null  int64  
 5   TRDATETIME    490513 non-null  object 
 6   amount        490513 non-null  float64
 7   trx_category  490513 non-null  object 
 8   target_flag   490513 non-null  int64  
 9   target_sum    490513 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 37.4+ MB


In [4]:
print("Total clients: ", len(raw_df.cl_id.unique()))

Total clients:  5000


Всего 5000 клиентов, случайным образом возьмем 1000 клиентов для тестирования

In [5]:
cl_ids_test = np.random.choice(raw_df.cl_id.unique(), size=1000, replace=False)
cl_ids_test_set = set(cl_ids_test)

In [6]:
# create transactions dataset for train
transactions_train = raw_df[~raw_df.cl_id.isin(cl_ids_test)].copy()   # то есть , то что не в списке cl_ids_test
print("Total transactions in train dataset: ", len(transactions_train))
# create transactions dataset for test
transactions_test = raw_df[raw_df.cl_id.isin(cl_ids_test)].copy()    # то что  в списке cl_ids_test
print("Total transactions in test dataset: ", len(transactions_test))

Total transactions in train dataset:  389648
Total transactions in test dataset:  100865


In [7]:
raw_df.cl_id.value_counts()

2143    784
5373    512
5630    501
4564    499
1261    485
       ... 
2620      1
4513      1
4502      1
9225      1
5224      1
Name: cl_id, Length: 5000, dtype: int64

## Домашняя работа

1. Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
2. Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
3. При агрегации (*pandas.DataFrame.groupby*) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

    - общая сумма транзакций по каждой из trx_category
    - общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
    - общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
    - возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.
3. **Обязательная часть**: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи
4. **Обязательная часть**: на большом количестве фичей применить Lasso регрессию и посмотреть, какие переменные получают 0 в качестве коэффициентов
5. **Обязательная часть**: на фичах из MCC кодов применить PCA и посмотреть, какие фичи получаются и что они могут объяснять (для этого посмотреть на коэффициенты в *sklearn.decomposition.pca.components_* после построения главных компонент)
6. **Обязательная часть**: с отобранными фичами и полученными компонентами обучить модель (тип алгоритма на свой вкус, можно начать с линейной) и померить качество на локальном тестовом наборе данных (локальная валидация), который создается в этом ноутбуке. Сравнить качество модели до добавления компонент/после добавления/только на главных компонентах. **Метрика оценки качества - ROC AUC**(https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
8. **Дополнительная часть**: поучаствовать в соревновании - загрузить предсказания на https://boosters.pro/champ_15. Для этого необходимо использовать все данные из файла rosbank_train.csv, на них делать feature engineering и обучениеб затем делать предсказания для клиентов из файла rosbank_test.csv, предварительно создав фичи по аналогии с train.
9. Задания принимаются в виде ноутбука с кодом/картинками выполненной обязательной части + указанием места в leaderboard при решении дополнительной

При возникновении вопросов и для отправки домашнего задания - egsachko@gmail.com или http://fb.com/sachkoe
    

pip install nbformat в начале нужно для плотли

## Разберемся с MCC

#### Найдем уникальные значения MCC групп, и расписать какие из кодов в какую группу входят.

In [8]:
mcc_cod=pd.read_excel('D:\\DDDrilll\\ds3-spring-2018\\kaggle\\rosssss\\Spravochnik_MCC_Cod.xlsx')
mcc_cod=mcc_cod.drop(labels=[0])
mcc_cod=mcc_cod.drop(columns=['Unnamed: 1','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10'])
mcc_cod.rename(columns={'Unnamed: 2':'Count'},inplace=True)

In [9]:
fig = px.funnel_area( names=mcc_cod['Count'].unique(), values=mcc_cod['Count'].value_counts())
fig.show()

In [10]:
uniq=mcc_cod['Count'].unique()

In [11]:
mcc_group={}
for i in uniq:
    code=[mcc_cod['Справочник МСС-кодов'][mcc_cod['Count']==f'{i}']]
    mcc_group[f'{i}']=code


In [12]:
mcc_top=raw_df['MCC'].value_counts(normalize=True)
mcc_top=mcc_top.loc[mcc_top>0.02].to_frame()
mcc_top['ind']=mcc_top.index.to_list()

fig=px.bar(mcc_top,x='ind',y='MCC')
fig.show()

посмотрим на распределения кол-ва денег по категориям транзакций

In [13]:
trx_cat=raw_df['trx_category'].value_counts()
currency_cat=raw_df['currency'].value_counts()

In [14]:
amount_trx=raw_df.groupby(['trx_category']).agg({'amount':'sum'})
amount_trx=amount_trx.reset_index()
fig=px.bar(amount_trx,x=amount_trx.trx_category,y=amount_trx.amount,width=1000, height=600)
fig.show()

для начала создадим сводную таблицу в которой представим кол-во потраченной суммы от кода валюты и типа транзакции и выведем графики

In [43]:
currency=raw_df['currency'].value_counts() 
currency=currency.loc[currency>500].index.to_list() 
# создали список сымых часто используемых валют и потом отсортировали в новый фрейм данные
# также учитывали и самые популярные категории транзакций 
curr_df=raw_df.loc[raw_df['currency'].isin(currency) & raw_df['trx_category'].isin(['POS','DEPOSIT','WD_ATM_ROS','WD_ATM_PARTNER'])]


In [44]:
for i in currency:
    # сводная таблица агрегированная нами
    # пересоздавать h обязательно , а то она читает плохо после первого цикла
    h=curr_df.groupby(['currency','trx_category']).agg({'amount':'sum'})
    h=h.reset_index()
    h=h.loc[h.currency==i]
    fig=px.bar(h,x=h.trx_category,y=h.amount,title=f'{i}',width=800, height=400)
    fig.show()

график показывает сколько потратили в каждой валюте и  категории транзакций кол-во денег. В итоге можно некоторвые выбросить, тк слишком малые суммы.  
в основнеом пользовались рублем , евро в транзакции 'POS', долларом в транзакции 'POS' вот их мы и будем использовать как идни из признаков.

теперь выделим дни недели по которым совершались операции

In [45]:
# напишем функцию, преобразующую дату в нужный формат
from datetime import datetime
def change_date(date):
    curr_date=datetime.strptime(date, '%d/%m/%Y')
    return curr_date.strftime('%Y-%m-%d')

# создадим новую переменную , к которой применим функцию замены даты
period_new=curr_df[['PERIOD']].copy(deep=True)
period_new['PERIOD_NEW']=period_new.PERIOD.apply(change_date)
# в столбце PERIOD_NEW время записано как YYYY-MM-DD
# в столбце PERIOD время записано как DD/MM/YYYY

In [46]:
period_new['PERIOD_NEW'] = pd.to_datetime(period_new['PERIOD_NEW'], errors='coerce')

In [47]:
period_new['Y']=period_new['PERIOD_NEW'].dt.year
period_new['M']=period_new['PERIOD_NEW'].dt.month
period_new['D']=period_new['PERIOD_NEW'].dt.day
period_new['target']=curr_df['target_flag']

Выделим клиентов еоторые продолжили пользоваться банком. Посмотрим как дата влияет на целевой показатель

In [87]:
period_new=period_new.loc[period_new['target']==1]
period_new['target'].value_counts()

1    300226
Name: target, dtype: int64

In [72]:
def count_n(i):
    '''Функция считает кол-во категорий в Serias'''
    from collections import Counter
    current=Counter(i)
    return pd.DataFrame(current, index=['amount']).transpose()

In [88]:
count_year=count_n(period_new['Y'])
fig=px.bar(
    x=count_year.index,
    y=count_year['amount'],
    labels = {'x': 'The Year', 'y': 'amount'},
    title="Количесво оставшихся клиентов в каждый год",
    )
fig.show()

Видно из графика, что больше всего клиентов продолжило пользоваться , которые в 2017 году совершали опепрации.
Около 260 тысяч операций было совершено за 2017.  
Тогда можно рассмотреть А в каком месяце тогда сосредоточены люди продолжающие пользоваться услугами банка

In [94]:

count_month=count_n(period_new['M'].loc[period_new['Y']==2017])
fig=px.bar(
    x=count_month.index,
    y=count_month['amount'],
    labels = {'x': 'The month', 'y': 'amount'},
    title="Количесво оставшихся клиентов в 2017 год",
    )
fig.show()

В основном операции приходятся на лето и конец весны. используем эти данные для дальнейшего создания фич

In [163]:
new_date=period_new['PERIOD_NEW']
new_date = pd.to_datetime(new_date).dt.dayofweek
# добавим необходимую форматированную дату к нашему фрейму
curr_df=curr_df.join(new_date)

# избавимся от старого
curr_df.drop(['PERIOD'],axis=1,inplace=True)

Теперь выделим время сутое и постараемся провести аналитику , зависит ли ночью или днем совершались покупки

In [188]:
import re
# С помощью регулярных выражений вытягиваем из столбца TRDATETIME информаци. о времени
time=[]
list_time=curr_df.TRDATETIME.to_list()
for i in list_time:
    patern=re.findall('(00|[0-9]|1[0-9]|2[0-3]):([0-9]|[0-5][0-9]):([0-9]|[0-5][0-9])$',i)
    time.extend(patern)
# преобразуем данные во фрейм
time=pd.DataFrame(time,columns=['hour','minute','second'])
#curr_df=curr_df.join(time['hour'])
t=time['hour'].to_numpy() 
# я специально пребразовал в ndarray тк он дает огромный выйгрыш по времени
# ссылка на статью 
# https://medium.com/nuances-of-programming/эффективное-итерирование-по-строкам-в-pandas-dataframe-43aa7d2588cb


In [190]:
# преобразуем значения в удобный формат
for i in range(0,len(t)):
    if t[i]=='00':
        t[i]=1
    else:
        t[i]=0
t=pd.DataFrame(t,columns=['night'])

In [189]:
from collections import Counter
cnt=Counter(time['hour'])
cnt_hour= pd.DataFrame(cnt, index=[0]).transpose()
fig=px.bar(cnt_hour, labels=dict(index="hour",))
fig.show()

Видно , что большинство операций совершалось в 12 часов ночи. Причем на несколько порядков больше чем других значений. поэтому выделим как отдельную фичу кол-во совершенных операций в 00 часов

In [193]:
curr_df.join(t['night'])
curr_df.dropna()

,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum,PERIOD_NEW,hour
2911,50,5411,type5,810,15SEP17:00:00:00,2373.0,POS,1,74592.37,4,00
2912,50,5812,type5,810,24SEP17:00:00:00,5955.0,POS,1,74592.37,4,00
2913,50,5661,type5,810,08JUL17:00:00:00,5599.0,POS,1,74592.37,5,00
2914,50,5732,type5,810,08JUL17:00:00:00,29600.0,POS,1,74592.37,5,00
2915,50,5812,type5,810,10SEP17:00:00:00,1038.0,POS,1,74592.37,4,00
...,...,...,...,...,...,...,...,...,...,...,...
462276,9598,5411,type1,810,03SEP17:00:00:00,435.0,POS,1,127.90,4,14
462277,9598,5411,type1,810,30AUG17:00:00:00,95.5,POS,1,127.90,1,00
462278,9603,5999,type1,810,16JAN17:00:00:00,809.0,POS,1,40716.74,6,00
462279,9598,5812,type1,810,30AUG17:00:00:00,99.0,POS,1,127.90,1,00


In [148]:
id_trx=raw_df.groupby(['cl_id','trx_category']).agg({'amount':'count'})
id_trx

amount
cl_id trx_category          
0     C2C_OUT              1
      DEPOSIT              1
      POS                  3
1     DEPOSIT              2
      POS                101
...                      ...
10213 POS                 62
      WD_ATM_PARTNER       1
10214 POS                107
10215 DEPOSIT              4
      POS                 20

[18361 rows x 1 columns]

In [149]:
amount_table=pd.DataFrame()

In [96]:
#df=pd.DataFrame.groupby(self=transactions_train ,by=['cl_id','MCC','channel_type']).agg({'': ''})
#df